# building cross-compiler GNU toolchain

Cross-compiler build is going using GNU [make](make.ipynb) tool, which is the minimal and still powerful automation tool for build control.

## target system configuration

configuration variables can be overriden in a command line:
```
cd msp430 ; make -f ../cross APP=cnc HW=pc104 TARGET=i686 binutils
```

### application

In [224]:
%%writefile Makefile

APP    ?= blink

Overwriting Makefile


### hardware device

Hardware device you are targetting, must be used for lot of configuration: GPIO ports, peripherial pingout,..

* `HW`
  * [msp430g2]()
  * [cm0pill]()
  * [cm1pill]()

In [225]:
%%writefile -a Makefile

HW     ?= msp430g2

Appending to Makefile


This special hint takes directory of `Makefile` in case of out of tree run like `cd msp430 ; make -C ../cross`

In [226]:
%%writefile -a Makefile

MKFDIR := $(dir $(abspath $(lastword $(MAKEFILE_LIST))))

Appending to Makefile


### target MCU/CPU/OS triplet

* `BUILD`
  - system name of your workstation OS in form of **system triplet**
    - `x86_64-pc-linux-gnu`
* `HOST`
  - triplet of OS on which package will run
    * for MCU targets we'll drop it using `HOST=BUILD`
* `TARGET`
  - triplet of MCU target
    * [msp430](msp430.ipynb) MSP430
    * [arm-none-eabi](arm-none-eabi.ipynb) Cortex-M
    * [i486-elf](i486-elf.ipynb) PC104 VortexSX/DX

In [227]:
%%writefile -a Makefile

include $(MKFDIR)/hw/$(HW)
include $(MKFDIR)/cpu/$(CPU)

Appending to Makefile


In [228]:
%%writefile hw/msp430g2

CPU    ?= msp430f2553

Overwriting hw/msp430g2


In [229]:
%%writefile cpu/msp430f2553

TARGET ?= msp430

Overwriting cpu/msp430f2553


## directory structure

* `Makefile`
* `hw/` hardware boards CPU & peripherials and specific tools defines
* `cpu/` CPU/MCU definitions

This set of directories will be created 

* `CWD` in the same dir as `Makefile` itself or directory selected by `make -C` option
* `GZ` downloaded archives
* `TMP` temp build directory
* `SRC` source code for every tool will be built
* `GNU` all compilers and tools will be pushed locally into this directory
  - such a scheme will not affect to whole `BUILD`/`HOST` system, all toolds will reside in a user `HOME`

In [230]:
%%writefile -a Makefile

CWD = $(CURDIR)
GZ  = $(HOME)/gz
TMP = $(CWD)/tmp
SRC = $(TMP)/src
GNU = $(CWD)/$(TARGET)

Appending to Makefile


In [231]:
%%writefile -a Makefile

all: dirs gz binutils gdb

Appending to Makefile


In [232]:
%%writefile -a Makefile

dirs:
	mkdir -p $(GZ) $(TMP) $(SRC) $(GNU)

Appending to Makefile


In [233]:
%%writefile .gitignore
*~
*.swp
tmp/
msp430/

Overwriting .gitignore


In [234]:
%%sh
make dirs ; ls -la ; ls -la tmp

mkdir -p /home/ponyatov/gz /home/ponyatov/jupy/cross/tmp /home/ponyatov/jupy/cross/tmp/src /home/ponyatov/jupy/cross/msp430
итого 60
drwxr-xr-x  7 ponyatov ponyatov  4096 май 29 22:26 .
drwxr-xr-x 12 ponyatov ponyatov  4096 май 29 21:43 ..
drwxr-xr-x  2 ponyatov ponyatov  4096 май 29 21:12 cpu
-rw-r--r--  1 ponyatov ponyatov 23494 май 29 22:26 cross.ipynb
-rw-r--r--  1 ponyatov ponyatov    22 май 29 22:27 .gitignore
drwxr-xr-x  3 ponyatov ponyatov  4096 май 29 21:12 hw
drwxr-xr-x  2 ponyatov ponyatov  4096 май 29 21:46 .ipynb_checkpoints
-rw-r--r--  1 ponyatov ponyatov   314 май 29 22:27 Makefile
drwxr-xr-x  2 ponyatov ponyatov  4096 май 29 22:11 msp430
drwxr-xr-x  3 ponyatov ponyatov  4096 май 29 22:11 tmp
итого 12
drwxr-xr-x 3 ponyatov ponyatov 4096 май 29 22:11 .
drwxr-xr-x 7 ponyatov ponyatov 4096 май 29 22:26 ..
drwxr-xr-x 2 ponyatov ponyatov 4096 май 29 22:11 src


## cross-compiler components

* `binutils` assembler, linker, object file tools
* `gdb` debugger
* `gcc` C/C++ compiler
  * `gmp` \ libraries
  * `mpfr`  required for
  * `mpc` / gcc build

## versions

In [235]:
%%writefile -a Makefile

BINUTILS_VER = 2.32
GDB_VER      = 8.3

Appending to Makefile


In [236]:
%%writefile -a Makefile

BINUTILS = binutils-$(BINUTILS_VER)
GDB      = gdb-$(GDB_VER)

Appending to Makefile


## predownload all archives

In [237]:
%%writefile -a Makefile

BINUTILS_GZ = $(BINUTILS).tar.xz
GDB_GZ      = $(GDB).tar.xz

Appending to Makefile


In [238]:
%%writefile -a Makefile

WGET = wget -c -P $(GZ) -q

gz: $(GZ)/$(BINUTILS_GZ) $(GZ)/$(GDB_GZ)

Appending to Makefile


In [239]:
%%sh
make gz ; ls -la ~/gz

make: Цель «gz» не требует выполнения команд.
итого 40316
drwxr-xr-x   2 ponyatov ponyatov     4096 май 29 21:37 .
drwxr-xr-x 181 ponyatov ponyatov    12288 май 29 21:13 ..
-rw-r--r--   1 ponyatov ponyatov 20774880 фев  2 20:38 binutils-2.32.tar.xz
-rw-r--r--   1 ponyatov ponyatov 20490300 май 11 22:47 gdb-8.3.tar.xz
-rw-r--r--   1 ponyatov ponyatov        0 окт 13  2017 .gitkeep


### archive unpack rules

In [240]:
%%writefile -a Makefile

$(SRC)/%/README: $(GZ)/%.tar.xz
	cd $(SRC) ; xzcat $< | tar -x && touch $@

Appending to Makefile


### configure scripts run

In [241]:
%%writefile -a Makefile

XPATH = PATH=$(GNU)/bin:$(PATH)

CFG = configure --disable-nls --prefix=$(GNU)

MAKE = $(XPATH) make

Appending to Makefile


most components can be built with use of full power of your computer, but `gcc` build can't be done in parallel even on i7/6G so `make gcc0` **will be run in single process** -- run in in background, and do some useful work while building: read docs, take a rest with tee, etc

In [242]:
%%writefile -a Makefile

CPU_CORES = $(shell grep processor /proc/cpuinfo|wc -l)

MAKEJ = $(MAKE) -j$(CPU_CORES)

Appending to Makefile


### [binutils](binutils.ipynb) : assembler, linker, object file tools

In [243]:
%%writefile -a Makefile

binutils: $(GNU)/bin/$(TARGET)-ld

CFG_BINUTILS += --target=$(TARGET)

$(GNU)/bin/$(TARGET)-ld: $(GZ)/$(BINUTILS_GZ) $(SRC)/$(BINUTILS)/README
	rm -rf $(TMP)/$(BINUTILS) ; mkdir $(TMP)/$(BINUTILS) ; cd $(TMP)/$(BINUTILS) ;\
	$(XPATH) $(SRC)/$(BINUTILS)/$(CFG) $(CFG_BINUTILS) && $(MAKEJ) && $(MAKE) install-strip

$(GZ)/$(BINUTILS_GZ):
	$(WGET) https://mirror.tochlab.net/pub/gnu/binutils/$(BINUTILS_GZ)

Appending to Makefile


In [244]:
%%sh
# make binutils ; ls -la tmp/src tmp msp430/bin

after `make` run `configure` will be run with required options:

`/home/dpon/jupy/msp430/tmp/src/binutils-2.32/configure --disable-nls --prefix=/home/user/jupy/msp430 --target=msp430`

detecting host/target

```
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking target system type... msp430-unknown-none
```

build requires some preinstalled developmen packages:
 * `sudo apt install`
   - `build-essential` `m4` `wget` `make` `gcc g++` `flex` `bison`
   - `xz` `bzip2`
   - `bc` `libncurses-dev` only for Linux kernel build

```
checking for gcc... gcc
checking for g++... g++
checking for bison... bison
checking for m4... m4
checking for flex... flex
```

`make -j4`

`make install-strip`

### [gdb](gdb.ipynb) : remote hardware debugger

In [245]:
%%writefile -a Makefile

gdb: $(GNU)/bin/$(TARGET)-gdb

CFG_GDB += $(CFG_BINUTILS) --enable-python

$(GNU)/bin/$(TARGET)-gdb: $(GZ)/$(GDB_GZ) $(SRC)/$(GDB)/README
	rm -rf $(TMP)/$(GDB) ; mkdir $(TMP)/$(GDB) ; cd $(TMP)/$(GDB) ;\
	$(XPATH) $(SRC)/$(GDB)/$(CFG) $(CFG_GDB) && $(MAKEJ) && $(MAKE) install

$(GZ)/$(GDB_GZ):
	$(WGET) http://ftp.gnu.org/gnu/gdb/$(GDB_GZ)

Appending to Makefile


In [246]:
%%sh
# make gdb ; ls -la msp430/bin/*gdb*

## cleaup after build

In [247]:
%%writefile -a Makefile

clean:
	rm -rf $(TMP)

distclean: clean
	rm -rf $(GNU)

Appending to Makefile
